In [88]:
import pandas as pd
import numpy as np
import seaborn as sns


In [89]:
houses_train = pd.read_csv('../data/processed_data.csv',keep_default_na=False)
houses_test = pd.read_csv('../data/cleaned_test_data.csv',keep_default_na=False)

In [90]:
houses_train.head()

,MSSubClass,LotFrontage,LotArea,Street,Utilities,YearBuilt,YearRemodAdd,MasVnrArea,BsmtExposure,BsmtUnfSF,...,Exter_Rating,Bsmt_rating,Basement_bathrooms,Aboveground_bathrooms,Kitchen_Rating,Fireplace_Rating,Total_Condtion,BsmtFinType1_rating,BsmtFinType2_rating,Bsmt_total_SF_rating
0,60,0.150685,0.033420,1.0,1.0,0.949275,0.883333,0.12250,0.25,0.064212,...,1.166667,1.55,0.333333,0.916667,0.222222,0.000000,2.0,0.125089,0.0,0.125089
1,20,0.202055,0.038795,1.0,1.0,0.753623,0.433333,0.00000,1.00,0.121575,...,0.833333,1.55,0.250000,0.666667,0.111111,0.200000,2.0,0.144401,0.0,0.144401
2,60,0.160959,0.046507,1.0,1.0,0.934783,0.866667,0.10125,0.50,0.185788,...,1.166667,1.55,0.333333,0.916667,0.222222,0.200000,2.0,0.086109,0.0,0.086109
3,70,0.133562,0.038561,1.0,1.0,0.311594,0.333333,0.00000,0.25,0.231164,...,0.833333,1.60,0.333333,0.333333,0.222222,0.266667,2.0,0.031892,0.0,0.031892
4,60,0.215753,0.060576,1.0,1.0,0.927536,0.833333,0.21875,0.75,0.209760,...,1.166667,1.55,0.333333,0.916667,0.222222,0.200000,2.0,0.116052,0.0,0.116052


In [91]:
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_squared_error
from sklearn.model_selection import train_test_split

In [92]:
X_train = houses_train.drop(['SalePrice'],axis=1)
y_train = np.log10(houses_train['SalePrice'])

In [93]:
y_train

0       5.319106
1       5.258877
2       5.349278
3       5.146128
4       5.397940
          ...   
1455    5.243038
1456    5.322219
1457    5.425697
1458    5.152670
1459    5.168792
Name: SalePrice, Length: 1460, dtype: float64

In [94]:
X_train_train, X_val, y_train_train, y_val = train_test_split(X_train.copy(), y_train.copy(),
                                                                 shuffle = True,
                                                                 random_state = 321,
                                                                 test_size=.15)

In [95]:
model = RandomForestRegressor(n_estimators=100, random_state=0)

In [96]:
model.fit(X_train_train,y_train_train)

RandomForestRegressor(random_state=0)

In [97]:
A = model.predict(X_val)

In [98]:
mean_squared_error(y_val,A)

0.004621148832456831

In [108]:
import xgboost

xgb_reg = xgboost.XGBRegressor(n_estimators=200,
                          max_depth=10,
                          learning_rate=.1)


xgb_reg.fit(X_train_train, y_train_train)

B = xgb_reg.predict(X_val)

In [109]:
mean_squared_error(y_val,B)

0.004903427542453027

In [110]:
C = xgb_reg.predict(X_train_train)
mean_squared_error(y_train_train,C)

7.607580280756367e-07